# A3 — Detecção de Evento de Escorregamento (Médio)

Detecte eventos de escorregamento a partir de trechos tácteis + corrente de motor (classificação binária). Otimize AUROC.

### Contexto
**Escorregamento (slip)** ocorre quando um gripper ou pé perde tração: forças tangenciais excedem a fricção e a área de contato cisalha. Matrizes tácteis (canais 0–5) capturam mudanças de pressão enquanto a corrente do motor (canais 6–7) reflete esforço extra para manter o movimento. Detectar cedo permite ajustar força de preensão, reprojetar colocação do pé ou modular a passada antes da falha.

Geramos trechos via [`common.data_utils.slip_detection_dataset`](common/data_utils.py): negativos exibem ruído nominal; positivos injetam pulsos tácteis locais e rampas ascendentes de corrente para emular slip incipiente.

### Por que métodos quânticos aqui?
Os indícios de escorregamento são sutis em janelas ruidosas. Comprimir estatísticas (energia, bandas espectrais, inclinações) em um embedding de baixa dimensão e testar kernels quânticos ou classificadores variacionais pode enriquecer fronteiras de decisão sob orçamento reduzido de qubits.

## Especificação de Entrada e Saída
**Entrada:** `X_train (N×W×8)`, `y_train (N,)`, `X_test (M×W×8)` com $W=64$ nos dados fornecidos  
**Saída:** `scores (M,)` escores reais (maior ⇒ mais provável escorregamento)  
**Tipo de retorno:** array float (ex.: `np.float64`)

Orientações de runtime/recursos: qubits ≤ 10, passos do otimizador ≤ 150; mantenha extração de características vetorizada.

### Dicas
- Canais tácteis 0–2 têm padrões de pulsos; considere potência de banda, wavelets ou energia em janela curta.
- Canal de corrente 7 cresce durante slip; inclinação (slope) ou soma acumulada são informativos.
- Correlacione pulsos tácteis com a inclinação da corrente para reduzir alarmes falsos.
- Ideia quântica: mapear estatísticas extraídas via [`common.quantum_utils.feature_map`](common/quantum_utils.py) e treinar um QSVM/VQC para maximizar AUROC.

_Orientação de baseline:_ [`common.baselines.baseline_slip`](common/baselines.py) tipicamente atinge AUROC ≈ 0.83–0.86 em seeds variadas, deixando margem acima do limiar público 0.82.

## Setup

In [ ]:
import numpy as np
from common import data_utils as du
from common import baselines as bl
from common import quantum_utils as qu
np.random.seed(1337)


## Baseline (referência)

In [ ]:
Xtr, ytr = du.slip_detection_dataset(n=500, w=64, seed=0, pos_rate=0.3)
Xte, yte = du.slip_detection_dataset(n=200, w=64, seed=1, pos_rate=0.3)
scores = bl.baseline_slip(Xtr, ytr, Xte)
from sklearn.metrics import roc_auc_score
au = roc_auc_score(yte, scores)
print(f"AUROC do baseline: {au:.3f}")


## Sua tarefa: implementar `solve(...)`

In [ ]:
def solve(Xtr, ytr, Xte):
    # TODO: extrair estatísticas vetorizadas → escalar → kernel quântico ou VQC (ver common.quantum_utils).
    return bl.baseline_slip(Xtr, ytr, Xte)

## Testes públicos (rápidos)

In [ ]:
from sklearn.metrics import roc_auc_score
Xtr, ytr = du.slip_detection_dataset(n=400, w=64, seed=10, pos_rate=0.3)
Xte, yte = du.slip_detection_dataset(n=200, w=64, seed=11, pos_rate=0.3)
scores = solve(Xtr, ytr, Xte)
au = roc_auc_score(yte, scores)
print("AUROC público:", au)
assert scores.shape == (len(yte),)
assert au >= 0.82, "AUROC público abaixo do limiar"
print("OK")


> Testes adicionais serão executados pelos organizadores com seeds/tamanhos diferentes.